In [7]:
#Code to calculate energetic particle H2,O2, and H2O production in Ganymede's polar cap
# The code uses as reference on yield and range thew work of Teolis et al. JGR 2017
# The energetic partcile fluxes are from Paranicas et al. GRL 2021
# Read in yield curve for energetic electrons and protonsenar = [1.0,10.,100.,1000.,10000.]
elecrange = [35000.,1100000.,60000000.,100000000.]
egrd = [10.,100.,1000.,10000.]
prorange = [350.,3500.,11000.,450000.,11000000.]
pgrd = [1.0,10.,100.,1000.,10000.]
eradyield = [0.03,0.005,0.0018,0.001]
pradyield = [0.5,0.8,2.2,0.13,0.02]
psptyield = [0.5,1.0,2.0,0.6,0.2]
#Import libraries
import pandas as pd
import numpy as np
import scipy as sp
from scipy import integrate
from scipy.interpolate import CubicSpline
import glob
import re
import math
import os
import plotly.express as px
import plotly.graph_objects as go
#Function calls for particle fluxes
def eeflx(Ee):
    j1=8.48e9
    E1=0.057
    gam1=0.81
    kap1=1.89
    j2=1.62e6
    E2=300.69
    gam2=0.0
    kap2=4.18
    er1=Ee/E1
    er2=Ee/E2
    coef11=gam1
    coef12=-gam1-kap1
    coef21=gam2
    coef22=-gam2-kap2
    je = (j1*er1**coef11*(1+er1)**coef12+j2*er2**coef21*(1+er2)**coef22)*3.1415
    return je
def epflx(Ep):
    j1=1.54e6
    E1=2.38
    gam1=0.0
    kap1=4.07
    j2=2.34e4
    E2=215.53
    gam2=0.0
    kap2=2.09
    er1=Ep/E1
    er2=Ep/E2
    coef11=gam1
    coef12=-gam1-kap1
    coef21=gam2
    coef22=-gam2-kap2
    jp = (j1*er1**coef11*(1+er1)**coef12+j2*er2**coef21*(1+er2)**coef22)*3.1415
    return jp
#Determine yield and energy flux for energetic electrons from 10keV to 10MeV
eeflxarray = np.zeros_like(egrd)
jEarray = np.zeros_like(egrd)
idx1 = len(egrd)
print(idx1)
for i in range(idx1):
    eeflxarray[i] = eeflx(egrd[i])*eradyield[i]
    jEarray[i] = eeflx(egrd[i])
print(eeflxarray)
#interpolation function for eeflx *  eyradyield
def log_interp1d(xx, yy, kind='linear'):
    logx = np.log10(xx)
    logy = np.log10(yy)
    lin_interp = sp.interpolate.interp1d(logx, logy, kind=kind)
    log_interp = lambda zz: np.power(10.0, lin_interp(np.log10(zz)))
    return log_interp
fe = log_interp1d(egrd,eeflxarray)
fef = log_interp1d(egrd,jEarray)
#perform integration 
idx2 = 9990
print(idx2)
egrdint= np.zeros(idx2)
print(egrdint)
EEyld = np.zeros_like(egrdint)
print(EEyld)
jEarrayint = np.zeros_like(egrdint)
print(jEarrayint)
e=10.0
print(e)

for i in range(0,idx2):
    EEyld[i] = fe(e)
    jEarrayint[i] = fef(e)*e
    egrdint[i] = e
    e=e+1.0
print(egrdint)
print(EEyld)
    
Eyldintval = integrate.simpson(EEyld,egrdint,even='avg')
Enflxval = integrate.simpson(jEarrayint,egrdint,even='avg')
print ( "Eyldintval=","{:.2e}".format(Eyldintval))
print("Enflxval=","{:.2e}".format(Enflxval))

#Determine yield and energy flux for protons from 1 keV to 10 MeV

epradflxarray = np.zeros_like(pgrd)
epsptflxarray = np.zeros_like(pgrd)
jEparray = np.zeros_like(pgrd)
idx3 = len(pgrd)
print(idx3)
for i in range(idx3):
    epradflxarray[i] = epflx(pgrd[i])*pradyield[i]
    epsptflxarray[i] = epflx(pgrd[i])*psptyield[i]
    jEparray[i] = epflx(pgrd[i])
print(epradflxarray)
print(epsptflxarray)
print(jEparray)
#interpolation function for epflx *  pradyield ; psptyield
def log_interpcs(xval,xx,yy):
    logxval= np.log10(xval)
    logx = np.log10(xx)
    logy = np.log10(yy)
    cs = CubicSpline(logx,logy)
    log_interp = cs(logxval)
    interp = np.power(10.0, log_interp)
    return interp
#perform integration 
idx4 = 99999
print(idx4)
pgrdint= np.zeros(idx4)
print(pgrdint)
Epradyld = np.zeros_like(pgrdint)
Epsptyld = np.zeros_like(pgrdint)
print(Epradyld)
print(Epsptyld)
jEparrayint = np.zeros_like(pgrdint)
print(jEparrayint)
e=1.0
print(e)

for i in range(0,idx4):
    Epradyld[i] = log_interpcs(e,pgrd,epradflxarray)
    Epsptyld[i] = log_interpcs(e,pgrd,epsptflxarray)
    jEparrayint[i] = log_interpcs(e,pgrd,jEparray)*e
    pgrdint[i] = e
    e=e+1.0
print(pgrdint)
print(Epradyld)
print(Epsptyld)
print(jEparrayint)
    
Epradyldintval = integrate.simpson(Epradyld,pgrdint,even='avg')
Epsptyldintval = integrate.simpson(Epsptyld,pgrdint,even='avg')
Epflxval = integrate.simpson(jEparrayint,pgrdint,even='avg')
print ( "Epradyldintval=","{:.2e}".format(Epradyldintval))
print ( "Epsptyldintval=","{:.2e}".format(Epsptyldintval))
print("Epflxval=","{:.2e}".format(Epflxval))


4
[1.78305978e+05 7.76164566e+03 2.05572939e+01 5.22303125e-03]
9990
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
10.0
[  10.   11.   12. ... 9997. 9998. 9999.]
[1.78305978e+05 1.56610790e+05 1.39118035e+05 ... 5.22866824e-03
 5.22678838e-03 5.22490938e-03]
Eyldintval= 3.27e+06
Enflxval= 4.94e+10
5
[6.16638155e+05 5.82007855e+04 7.29165328e+04 2.57138540e+02
 4.62443610e-01]
[6.16638155e+05 7.27509819e+04 6.62877571e+04 1.18679326e+03
 4.62443610e+00]
[1.23327631e+06 7.27509819e+04 3.31438785e+04 1.97798877e+03
 2.31221805e+01]
99999
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
1.0
[1.0000e+00 2.0000e+00 3.0000e+00 ... 9.9997e+04 9.9998e+04 9.9999e+04]
[6.16638155e+05 1.13782215e+05 6.82876003e+04 ... 2.33600994e+00
 2.33609579e+00 2.33618164e+00]
[6.16638155e+05 1.61599042e+05 1.03405943e+05 ... 3.52845954e-01
 3.52846588e-01 3.52847223e-01]
[1233276.31073755  622578.88436234  534624.26375627 ...   24258.920